# **SETUP**

## Import

In [1]:
from __future__ import division
import pickle
import numpy as np
import os
from openrec.tf1.legacy import ImplicitModelTrainer
from openrec.tf1.legacy.utils import ImplicitDataset
from openrec.tf1.legacy.utils.evaluators import ImplicitEvalManager
from openrec.tf1.legacy.recommenders import CML, BPR
from openrec.tf1.legacy.utils.evaluators import AUC
from openrec.tf1.legacy.utils.samplers import PairwiseSampler

## Init

In [2]:
auc_results = []
recall_results = []

seed = 76424236
np.random.seed(seed=seed)

folder_name = f"./Dataset/"

if os.path.exists(folder_name) == False:
    os.makedirs(folder_name)

## Load training set

In [3]:
raw_data = dict()
raw_data['train_data'] = np.load(folder_name + "training_arr.npy")
raw_data['test_data_pos'] = np.load(folder_name + "biased-test_arr_pos.npy")
raw_data['test_data_neg'] = np.load(folder_name + "biased-test_arr_neg.npy")
raw_data['max_user'] = 15401
raw_data['max_item'] = 1001
batch_size = 8000
test_batch_size = 1000
display_itr = 1000

train_dataset = ImplicitDataset(raw_data['train_data'], raw_data['max_user'], raw_data['max_item'], name='Train')
test_dataset_pos = ImplicitDataset(raw_data['test_data_pos'], raw_data['max_user'], raw_data['max_item'])
test_dataset_neg = ImplicitDataset(raw_data['test_data_neg'], raw_data['max_user'], raw_data['max_item'])

## Define Model

In [ ]:
#Code to avoid tf using cached embeddings
import tensorflow as tf
tf.compat.v1.reset_default_graph()

cml_model = CML(batch_size=batch_size, max_user=train_dataset.max_user(), max_item=train_dataset.max_item(),
    dim_embed=50, l2_reg=0.001, opt='Adam', sess_config=None)
sampler = PairwiseSampler(batch_size=batch_size, dataset=train_dataset, num_process=4)
model_trainer = ImplicitModelTrainer(batch_size=batch_size, test_batch_size=test_batch_size,
                                     train_dataset=train_dataset, model=cml_model, sampler=sampler,
                                     eval_save_prefix=folder_name+"yahoo",
                                     item_serving_size=500)
auc_evaluator = AUC()

cml_model.load(folder_name+"cml-yahoo")

# **DEFINE FUNCTION**

In [5]:
def eq(infilename, infilename_neg, trainfilename, gamma=-1.0, K=30):

    # Load the pickles
    infile = open(infilename, 'rb')
    infile_neg = open(infilename_neg, 'rb')
    P = pickle.load(infile)
    infile.close()
    P_neg = pickle.load(infile_neg)
    infile_neg.close()
    NUM_NEGATIVES = P["num_negatives"]
    
    # Merge the two dictionaries
    for theuser in P["users"]:
        neg_items = list(P_neg["user_items"][theuser][NUM_NEGATIVES:])
        neg_scores = list(P_neg["results"][theuser][NUM_NEGATIVES:])
        P["user_items"][theuser] = list(neg_items) + list(P["user_items"][theuser][NUM_NEGATIVES:])
        P["results"][theuser] = list(neg_scores) + list(P["results"][theuser][NUM_NEGATIVES:])
    
    Zui = dict()
    Ni = dict()

    # Compute the frequency of each item in the training set
    trainset = np.load(trainfilename)
    for i in trainset['item_id']:
        if i in Ni:
            Ni[i] += 1
        else:
            Ni[i] = 1
    del trainset

    # Count the number of users with at least one positive item in the test set
    # (not the smartest way)
    nonzero_user_count = 0
    for theuser in P["users"]:
        pos_items = P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]
        for pos_item in pos_items:
            if pos_item in Ni:
                nonzero_user_count += 1
                break

    # Compute the recommendations
    for theuser in P["users"]:
        all_scores = np.array(P["results"][theuser])
        pos_items = P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]
        pos_scores = P["results"][theuser][len(P_neg["results"][theuser][NUM_NEGATIVES:]):]
        for i, pos_item in enumerate(pos_items):
            pos_score = pos_scores[i]
            Zui[(theuser, pos_item)] = float(np.sum(all_scores > pos_score))

    # Compute the scores using AUC and compute the recall
    sum_user_auc = 0.0
    sum_user_recall = 0.0
    for theuser in P["users"]:
        numerator_auc = 0.0
        numerator_recall = 0.0
        denominator = 0.0
        for theitem in P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]:
            if theitem not in Ni:
                continue
            pui = np.power(Ni[theitem], (gamma + 1) / 2.0)
            numerator_auc += (1 - Zui[(theuser, theitem)] / len(P["user_items"][theuser])) / pui
            if Zui[(theuser, theitem)] < K:
                numerator_recall += 1.0 / pui
            denominator += 1 / pui
        if denominator > 0:
            sum_user_auc += numerator_auc / denominator
            sum_user_recall += numerator_recall / denominator

    return {
        "auc"       : sum_user_auc / nonzero_user_count,
        "recall"    : sum_user_recall / nonzero_user_count
    }

# **TEST**

## Generate Raw Results

In [ ]:
model_trainer._eval_manager = ImplicitEvalManager(evaluators=[auc_evaluator])
model_trainer._num_negatives = 200
model_trainer._exclude_positives([train_dataset, test_dataset_pos, test_dataset_neg])
model_trainer._sample_negatives(seed=10)

model_trainer._eval_save_prefix = folder_name+"cml-yahoo-test-pos-biased"
model_trainer._evaluate_partial(test_dataset_pos)

model_trainer._eval_save_prefix = folder_name+"cml-yahoo-test-neg-biased"
model_trainer._evaluate_partial(test_dataset_neg)

## Compute results

In [ ]:
eq(folder_name+"cml-yahoo-test-pos-biased_evaluate_partial.pickle", folder_name+"cml-yahoo-test-neg-biased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=1.5)